In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 26 13:03:59 2024

@author: joy
"""

import hashlib
import pandas as pd
from datetime import datetime


df = pd.read_csv("/content/活頁簿1.csv",encoding='Utf-8')  # 或者使用 pd.read_csv('data.csv') 如果是 CSV 文件


'''USER_ID'''
df['USER_ID_HASH'] = df.apply(
    lambda row: int(hashlib.sha256(f"{row['username']}{row['start_time_for_database']}".encode()).hexdigest(), 16),
    axis=1
)


'''Label Encoding'''
df['gender'] = df['gender'].map({'男生': 1, '女生': 2})
df['right_eye_shine'] = df['right_eye_shine'].map({'有': 1, '無': 0})
df['left_eye_shine'] = df['left_eye_shine'].map({'有': 1, '無': 0})
df['use_situation1'] = df['use_situation1'].map({'是': 1, '否': 0})
df['use_situation3'] = df['use_situation3'].map({'是': 1, '否': 0})
df['habit1'] = df['habit1'].map({'是': 1, '否': 0})
df['habit2'] = df['habit2'].map({'有': 1, '無': 0})
df['question_5'] = df['question_5'].map({'是': 1, '否': 0})
df['question_6'] = df['question_6'].map({'是': 1, '否': 0})

'''二進制編碼'''
binary_map = {"無": "00","近視": "01","遠視": "10"}
df['right_eye_condition'] = df['right_eye_condition'].map(binary_map)
df['left_eye_condition'] = df['left_eye_condition'].map(binary_map)


'''One-Hot Encoding'''
possible_columns = {
    "use_situation2": ["3小時以內","3至6小時","6-9小時",
                       "9-12小時","12小時以上" ],
    "use_situation_value4": ["電腦自動調整",
                            "不常調整",
                            "每次使用都會調整"
                             ],
    "use_situation_value5": ["僅室內共用燈光",
                            "僅室內專用燈光",
                            "室內共用與專用燈光皆有",
                            "戶外",
                            "光線明顯不足之環境",
                            "其他" ],
    "habit2": ["無","半年一次","一年一次","更頻繁"],
    "habit3": ["低於4小時","4至6小時","6至8小時","高於8小時"],
    "habit4": ["0或1次","2或3次","4或5次","6次以上"],
    "habit5": ['無休息',"1小時內","1-2小時","2-3小時",
               "3-4小時","4-5小時", "5小時以上"],
    "habit6": ["10分鐘內","11-30分鐘","31-60分鐘","60分鐘以上"],
    "habit7": ["眼部運動",'閉目養神','其他',
               '閉目養神, 眼部運動','閉目養神, 其他','眼部運動, 其他',
               '閉目養神, 眼部運動, 其他'],
    "question_1": ["電腦", "手機", "平板","其他"],
    "question_2": ["工作/實習用途", "聆聽線上課程","完成學校作業", "打電腦遊戲",
                   "觀看影音串流平台(如youtube)","回復訊息文字","其他"],
    "question_3": ["僅室內共用燈光","僅室內專用燈光", "戶外", "光線明顯不足之環境","其他" ],
    "question_4": ["無", "配戴眼鏡", "配戴隱形眼鏡"]
}

# 創建副本以避免直接修改原始數據
processed_data = df.copy()

# 替換欄位
for col, possible_values in possible_columns.items():
    # 獲取當前數據的獨熱編碼
    one_hot = pd.get_dummies(df[col], prefix=col)

    # 添加缺失的類別列（以 0 填充）
    for value in possible_values:
        col_name = f"{col}_{value}"
        if col_name not in one_hot.columns:
            one_hot[col_name] = 0

    # 按照可能值的順序排列列
    one_hot = one_hot[[f"{col}_{value}" for value in possible_values]]

    # 刪除原始欄位並將新編碼欄位插入
    processed_data = processed_data.drop(columns=[col]).reset_index(drop=True)
    processed_data = pd.concat([processed_data, one_hot], axis=1)

''' 計算年齡 '''
def calculate_age(birthday):
    # 將生日轉換為 datetime 格式
    birthday = datetime.strptime(str(birthday), "%Y%m%d")
    # 獲取當前日期
    today = datetime.today()
    # 計算年齡
    age = today.year - birthday.year - ((today.month, today.day) < (birthday.month, birthday.day))
    return age

processed_data["birthday"] = processed_data["birthday"].apply(calculate_age)
processed_data = processed_data.rename(columns={"birthday": "age"})


processed_data = processed_data.drop(['username', 'name','start_time_for_database','line_token'], axis=1, errors='ignore')


# 將目標欄位移動到最前面
column_to_move = "USER_ID_HASH"
processed_data = processed_data[[column_to_move] + [col for col in processed_data.columns if col != column_to_move]]


processed_data = processed_data.astype(float)


# 輸出為新的 CSV 文件
processed_data.to_csv('mandy_encoding.csv', index=False, encoding='utf-8-sig')




In [ ]:
df

,username,start_time_for_database,name,birthday,gender,right_eye_condition,right_eye_degree,right_eye_shine,right_eye_shine_degree,left_eye_condition,...,question_3,question_4,question_5,question_6,question_7,question_8,question_9,question_10,question_11,USER_ID_HASH
0,RYAN,2.024110e+11,趙永鈞,20021203,1,01,-300,1,100,01,...,僅室內共用燈光,配戴眼鏡,1,1,6,5,10,10,10,2060711433258662556248400069270730989379058302...
1,RYAN,2.024110e+11,趙永鈞,20021203,1,01,-300,0,100,01,...,僅室內共用燈光,配戴眼鏡,1,1,6,5,10,10,10,2060711433258662556248400069270730989379058302...
2,RYAN,2.024110e+11,趙永鈞,20021203,1,01,-300,0,100,01,...,僅室內共用燈光,配戴眼鏡,1,1,6,5,10,10,10,2060711433258662556248400069270730989379058302...
3,RYAN,2.024110e+11,趙永鈞,20021203,1,01,-300,1,100,01,...,僅室內共用燈光,配戴眼鏡,1,1,6,5,10,10,10,2060711433258662556248400069270730989379058302...
4,RYAN,2.024110e+11,趙永鈞,20021203,1,01,-300,1,100,01,...,僅室內共用燈光,配戴眼鏡,1,1,6,5,10,10,10,2060711433258662556248400069270730989379058302...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,MandyPC01,2.024110e+11,蔡佳芯,20021230,2,01,0,1,150,01,...,室內共用與專用燈光皆有,無,0,1,9,9,10,10,10,9440000854056047066789961949673860135328812288...
716,MandyPC01,2.024110e+11,蔡佳芯,20021230,2,01,0,1,150,01,...,室內共用與專用燈光皆有,無,0,1,9,9,10,10,10,9440000854056047066789961949673860135328812288...
717,MandyPC01,2.024110e+11,蔡佳芯,20021230,2,01,0,1,150,01,...,室內共用與專用燈光皆有,無,0,1,9,9,10,10,10,9440000854056047066789961949673860135328812288...
718,MandyPC01,2.024110e+11,蔡佳芯,20021230,2,01,0,1,150,01,...,室內共用與專用燈光皆有,無,0,1,9,9,10,10,10,9440000854056047066789961949673860135328812288...


In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib

In [8]:
# === Step 1: 讀取資料 ===
data = pd.read_csv('/content/all_plot_cleandata.csv')
data = data.dropna()
data

,USER_ID_HASH,age,gender,right_eye_condition,right_eye_degree,right_eye_shine,right_eye_shine_degree,left_eye_condition,left_eye_degree,left_eye_shine,...,question_3_光線明顯不足之環境,question_3_其他,question_4_無,question_4_配戴眼鏡,question_4_配戴隱形眼鏡,Transition,time_diff_to_first_exhausted,distance_moving_avg,brightness_moving_avg,distance_variance
0,3.150000e+74,22.0,1.0,1.0,-275.0,1.0,0.0,1.0,-300.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0,21.0,1.384000,160.000000,0.010588
1,3.150000e+74,22.0,1.0,1.0,-275.0,1.0,0.0,1.0,-300.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0,23.5,1.466000,160.000000,0.013448
2,3.150000e+74,22.0,1.0,1.0,-275.0,1.0,0.0,1.0,-300.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0,23.5,1.433667,157.333333,0.009860
3,3.150000e+74,22.0,1.0,1.0,-275.0,1.0,0.0,1.0,-300.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0,23.5,1.458750,157.750000,0.009090
4,3.150000e+74,22.0,1.0,1.0,-275.0,1.0,0.0,1.0,-300.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0,23.5,1.420800,157.000000,0.014019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,1.110000e+77,21.0,2.0,1.0,0.0,1.0,150.0,1.0,-75.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0,0.0,1.000000,10.000000,0.000000
2250,1.110000e+77,21.0,2.0,1.0,0.0,1.0,150.0,1.0,-75.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0,0.0,1.000000,10.000000,0.000000
2251,1.110000e+77,21.0,2.0,1.0,0.0,0.0,150.0,1.0,-75.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0,0.0,1.000000,10.000000,0.000000
2252,1.110000e+77,21.0,2.0,1.0,0.0,0.0,150.0,1.0,-75.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0,0.0,1.000000,10.000000,0.000000


資料預處理(增加特徵工程)

In [12]:
# === Step 2: 新增轉變標籤 ===
data['Transition'] = data.groupby('USER_ID_HASH')['Exhausted_state'].transform(
    lambda x: (x.shift() == 0) & (x == 1)
).astype(int)


# === Step 3: 計算首次轉變時間差 ===
def calculate_time_diff(df):
    start_time = df['time'].iloc[0]
    if df['Transition'].sum() > 0:  # 如果有轉變
        transition_time = df.loc[df['Transition'] == 1, 'time'].iloc[0]
        df['time_diff_to_first_exhausted'] = transition_time - start_time
    else:
        df['time_diff_to_first_exhausted'] = None
    return df

data = data.groupby('USER_ID_HASH', group_keys=False, as_index=False).apply(calculate_time_diff)

# === Step 4: 計算移動特徵 ===
def add_moving_features(df):
    df['distance_moving_avg'] = df.groupby('USER_ID_HASH')['distance'].transform(
        lambda x: x.rolling(window=5, min_periods=1).mean()
    )
    df['cumulative_distance'] = df.groupby('USER_ID_HASH')['distance'].cumsum()
    df['cumulative_time'] = df.groupby('USER_ID_HASH')['time'].transform(lambda x: x - x.min())
    return df

data = add_moving_features(data)

# === Step 5: 過濾資料 ===
def filter_first_transition(df):
    if df['Transition'].sum() == 0:  # 若無轉變，保留所有資料
        return df
    first_transition_index = df[df['Transition'] == 1].index[0]
    return df[df.index <= first_transition_index]

filtered_data = data.groupby('USER_ID_HASH', group_keys=False).apply(filter_first_transition).reset_index(drop=True)

<ipython-input-12-ec2172dd228f>:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = data.groupby('USER_ID_HASH', group_keys=False, as_index=False).apply(calculate_time_diff)
<ipython-input-12-ec2172dd228f>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('USER_ID_HASH', group_keys=False, as_index=False).apply(calculate_time_diff)
<ipython-input-12-ec2172dd228f>:37: DeprecationWarning: DataFrameGroupBy.apply operated on the gr

填補time_diff_to_first_exhausted

In [13]:
# 計算相關矩陣
correlation_matrix = filtered_data.corr()
correlation_with_target = correlation_matrix['time_diff_to_first_exhausted'].drop('time_diff_to_first_exhausted')
print("與 time_diff_to_first_exhausted 的相關係數：")
print(correlation_with_target)

與 time_diff_to_first_exhausted 的相關係數：
USER_ID_HASH             0.634583
age                     -0.078924
gender                   0.078924
right_eye_condition      0.642544
right_eye_degree         0.207007
                           ...   
brightness_moving_avg   -0.778665
distance_variance       -0.172753
cumulative_distance      0.539874
cumulative_time          0.535529
log_time_diff            0.983577
Name: time_diff_to_first_exhausted, Length: 106, dtype: float64


In [14]:
# 用KNNImputer 填補time_diff_to_first_exhausted
features_to_impute = ['time_diff_to_first_exhausted', 'brightness_x' , 'distance_ratio', 'brightness_y',
                      'blink_y','question_7','question_9','question_10','question_2_其他',
                      'question_2_完成學校作業','brightness_moving_avg','distance_variance',
                      ]
imputer = KNNImputer(n_neighbors=10)#建立 KNNImputer 模型
filtered_data[features_to_impute] = imputer.fit_transform(filtered_data[features_to_impute]) #使用 KNNImputer 填補缺失值


特徵選擇與清理

In [ ]:
# === Step 6: 特徵選擇與清理 ===
feature_cols = ['USER_ID_HASH',
 'age','gender',
 'right_eye_condition','right_eye_degree','right_eye_shine','right_eye_shine_degree',
 'left_eye_condition','left_eye_degree','left_eye_shine','left_eye_shine_degree',
 'eye_situation_value1','eye_situation_value2','eye_situation_value3','eye_situation_value4','eye_situation_value5',
 'use_situation1','use_situation3','habit1',
 'distance','brightness_x','blink_x','state','Exhausted_state','time',
 'distance_ratio','brightness_y','blink_y','blink_num',
 'question_5','question_6','question_7','question_8','question_9','question_10','question_11',
 'use_situation2_3小時以內','use_situation2_3至6小時','use_situation2_6-9小時','use_situation2_9-12小時','use_situation2_12小時以上',
 'use_situation_value4_電腦自動調整','use_situation_value4_不常調整','use_situation_value4_每次使用都會調整',
 'use_situation_value5_僅室內共用燈光','use_situation_value5_僅室內專用燈光','use_situation_value5_室內共用與專用燈光皆有','use_situation_value5_戶外','use_situation_value5_光線明顯不足之環境','use_situation_value5_其他',
 'habit2_無','habit2_半年一次','habit2_一年一次','habit2_更頻繁',
 'habit3_低於4小時','habit3_4至6小時','habit3_6至8小時','habit3_高於8小時',
 'habit4_0或1次','habit4_2或3次','habit4_4或5次','habit4_6次以上',
 'habit5_無休息','habit5_1小時內','habit5_1-2小時','habit5_2-3小時','habit5_3-4小時','habit5_4-5小時','habit5_5小時以上',
 'habit6_10分鐘內','habit6_11-30分鐘','habit6_31-60分鐘','habit6_60分鐘以上',
 'habit7_眼部運動','habit7_閉目養神','habit7_其他','habit7_閉目養神, 眼部運動','habit7_閉目養神, 其他','habit7_眼部運動, 其他','habit7_閉目養神, 眼部運動, 其他',
 'question_1_電腦','question_1_手機','question_1_平板','question_1_其他',
 'question_2_工作/實習用途','question_2_聆聽線上課程', 'question_2_完成學校作業','question_2_打電腦遊戲','question_2_觀看影音串流平台(如youtube)','question_2_回復訊息文字','question_2_其他',
 'question_3_僅室內共用燈光','question_3_僅室內專用燈光','question_3_戶外','question_3_光線明顯不足之環境','question_3_其他',
 'question_4_無','question_4_配戴眼鏡','question_4_配戴隱形眼鏡',
 'Transition','time_diff_to_first_exhausted','distance_moving_avg','brightness_moving_avg','distance_variance']


In [18]:
correlation_matrix = data.corr()
target_variable = 'time_diff_to_first_exhausted'

# 對數變換目標值
data['log_time_diff'] = np.log1p(data[target_variable] + 1e-5)

selected_features = correlation_matrix[target_variable][correlation_matrix[target_variable].abs() > 0.2].index.tolist()
selected_features.remove(target_variable)

In [25]:
len(selected_features)

42

In [19]:
# === 檢查 ===
data = data.dropna(subset=selected_features + ['Transition'])  # 移除缺失值
print(data.isnull().sum())  # 檢查每列缺失值
print((data == float('inf')).sum())  # 檢查正無窮
print((data == float('-inf')).sum())

USER_ID_HASH             0
age                      0
gender                   0
right_eye_condition      0
right_eye_degree         0
                        ..
brightness_moving_avg    0
distance_variance        0
cumulative_distance      0
cumulative_time          0
log_time_diff            0
Length: 107, dtype: int64
USER_ID_HASH             0
age                      0
gender                   0
right_eye_condition      0
right_eye_degree         0
                        ..
brightness_moving_avg    0
distance_variance        0
cumulative_distance      0
cumulative_time          0
log_time_diff            0
Length: 107, dtype: int64
USER_ID_HASH             0
age                      0
gender                   0
right_eye_condition      0
right_eye_degree         0
                        ..
brightness_moving_avg    0
distance_variance        0
cumulative_distance      0
cumulative_time          0
log_time_diff            0
Length: 107, dtype: int64


In [21]:
# === 切分X、Y值 ===
X = data[selected_features]
y = data['time_diff_to_first_exhausted']

In [22]:
# === Step 7: 切分資料集 ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Step 3: 標準化特徵 ===
scaler = StandardScaler()

# 對訓練集進行標準化
X_train_scaled = scaler.fit_transform(X_train)

# 使用相同的標準對測試集進行轉換
X_test_scaled = scaler.transform(X_test)

In [23]:
# === Step 4: 訓練回歸模型 ===
regressor = SVR(kernel='rbf')  # 使用 RBF 核
regressor.fit(X_train_scaled, y_train)

SVR()

In [24]:
# 預測並評估
y_pred = regressor.predict(X_test_scaled)
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred))
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2 Score:", r2_score(y_test, y_pred))

Mean Absolute Error (MAE): 0.28169264427253043
Mean Squared Error (MSE): 0.24333914289463807
Root Mean Squared Error (RMSE): 0.49329417480306625
R^2 Score: 0.9980222281738349


In [26]:
# 儲存模型
joblib.dump(regressor, 'svr_model.joblib')  # 將模型保存到文件中
joblib.dump(scaler, 'scaler.joblib')        # 保存標準化對象

print("模型已成功保存！")

模型已成功保存！


萊恩研究中

In [ ]:
# 匯入必要的模組
import pandas as pd

# 讀取 CSV 檔案
# 將 'your_file.csv' 替換為您的檔案名稱或路徑
file_path = '/content/活頁簿1.csv'  # 替換為您的檔案路徑
data = pd.read_csv(file_path)

# 顯示資料基本資訊
print("資料欄位名稱：")
print(data.columns.tolist())

# 資料分布型態
print("\n資料分布型態：")
print(data.describe(include='all').transpose())

# 缺失值概況
print("\n缺失值概況：")
print(data.isnull().sum())


資料欄位名稱：
['username', 'start_time_for_database', 'name', 'birthday', 'gender', 'right_eye_condition', 'right_eye_degree', 'right_eye_shine', 'right_eye_shine_degree', 'left_eye_condition', 'left_eye_degree', 'left_eye_shine', 'left_eye_shine_degree', 'eye_situation_value1', 'eye_situation_value2', 'eye_situation_value3', 'eye_situation_value4', 'eye_situation_value5', 'use_situation1', 'use_situation2', 'use_situation3', 'use_situation_value4', 'use_situation_value5', 'habit1', 'habit2', 'habit3', 'habit4', 'habit5', 'habit6', 'habit7', 'line_token', 'distance', 'brightness_x', 'blink_x', 'state', 'Exhausted_state', 'time', 'distance_ratio', 'brightness_y', 'blink_y', 'blink_num', 'question_1', 'question_2', 'question_3', 'question_4', 'question_5', 'question_6', 'question_7', 'question_8', 'question_9', 'question_10', 'question_11', 'USER_ID_HASH']

資料分布型態：
                         count unique  \
username                   720      4   
start_time_for_database  720.0    NaN   
name   